In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std

from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from scipy.stats import mode

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 5.1 MB/s 


In [ ]:
#import pyeeg
from tensorflow import keras 

In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100
from keras.utils import np_utils

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way_sub_466_trial_3.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way_sub_480_trial_2.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119929,-0.050405,-0.033532,0.013069,0.030343,0.028335,0.016684,0.073731,0.016684,-0.012642,-0.014651,...,-0.054423,-0.057636,-0.064868,-0.035139,0.001418,0.055251,0.019496,-0.035943,-0.035943,0
119930,-0.006616,-0.006616,0.037173,0.071320,0.102655,0.082970,0.023514,0.014676,-0.006616,0.031147,...,0.141222,0.170548,0.170950,0.155684,0.146444,0.144034,0.141222,0.138008,0.160907,0
119931,-0.188914,0.020942,-0.246959,-0.030852,-0.049159,0.021835,0.018263,-0.022815,-0.025941,0.114707,...,-0.088898,-0.214811,0.074969,0.055769,-0.110776,-0.067465,-0.246066,-0.050498,-0.079521,0
119932,0.046011,0.063285,0.033557,-0.000590,0.007846,0.027933,0.023514,0.025121,0.034361,0.044806,...,0.019496,0.043199,0.046814,0.040387,0.033557,0.030745,0.014274,-0.038353,-0.041165,4


In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 14.9 MB/s 


In [ ]:
x_dt=transfer_set_with_tri_trained_sub_labeled_dt.drop([3000],axis=1)
y_dt=transfer_set_with_tri_trained_sub_labeled_dt[3000]

In [ ]:
y_dt_=np_utils.to_categorical(y_dt)

In [ ]:
y_dt_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
x_dt.shape

(119934, 3000)

In [ ]:
x_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119929,-0.050405,-0.033532,0.013069,0.030343,0.028335,0.016684,0.073731,0.016684,-0.012642,-0.014651,...,-0.002197,-0.054423,-0.057636,-0.064868,-0.035139,0.001418,0.055251,0.019496,-0.035943,-0.035943
119930,-0.006616,-0.006616,0.037173,0.071320,0.102655,0.082970,0.023514,0.014676,-0.006616,0.031147,...,0.131179,0.141222,0.170548,0.170950,0.155684,0.146444,0.144034,0.141222,0.138008,0.160907
119931,-0.188914,0.020942,-0.246959,-0.030852,-0.049159,0.021835,0.018263,-0.022815,-0.025941,0.114707,...,0.041481,-0.088898,-0.214811,0.074969,0.055769,-0.110776,-0.067465,-0.246066,-0.050498,-0.079521
119932,0.046011,0.063285,0.033557,-0.000590,0.007846,0.027933,0.023514,0.025121,0.034361,0.044806,...,0.000615,0.019496,0.043199,0.046814,0.040387,0.033557,0.030745,0.014274,-0.038353,-0.041165


In [ ]:
x_dt_rshp=np.array(x_dt).reshape(119934,3000,1)

In [ ]:
x_dt_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[-0.18891398],
        [ 0.02094201],
        [-0.24695925],
        ...,
        [-0.24606625],
        [-0.05049833],
        [-0.07952097]],

       [[ 0.04601093],
        [ 0.06328549],
        [ 0.03355717],
        ...,
        [ 0.01427393],
        [-0.03835323],
        [-0.04116537]],

       [[-0.25008478],
        [ 0.11783296],
        [ 0.12854902],
        ...,
        [ 0.56478157],
        [ 0.60853877],
        [ 0.60094824]]])

In [ ]:
import numpy as np
np.random.seed(999)

import tensorflow
tensorflow.random.set_seed(999)

import random
#random.seed(999)

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-gb89we4_
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-gb89we4_
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=74f431ff187060965a3d22e343d10f0686d336fc5ae60788ad025ba30002afe7
  Stored in directory: /tmp/pip-ephem-wheel-cache-k3qwfkrl/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

In [ ]:
import keras_tuner as kt
optimizer=Adam(lr=1e-4)

In [ ]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
      model=model_b(verbose=VBS)
      optimizer = Adam(lr=1e-4)
      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
      optimizer = Adam(lr=1e-4)
      return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            x_dt_rshp, y_dt_,
            epochs=hp.Choice("epochs", [40, 50, 60, 70])
        )

In [ ]:
tuner = kt.RandomSearch(
    MyHyperModel(),
    objective="loss",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    #directory="my_dir",
    #project_name="tune_hypermodel",
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
tuner.search(x_dt_rshp, y_dt_)

Trial 3 Complete [00h 48m 35s]
loss: 0.07456402108073235

Best loss So Far: 0.07456402108073235
Total elapsed time: 02h 06m 02s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
epochs: 50
Score: 0.07456402108073235
Trial summary
Hyperparameters:
epochs: 40
Score: 0.08825397491455078
Trial summary
Hyperparameters:
epochs: 40
Score: 0.088563472032547


Trial 1:

tuner.search():

Trial 3 Complete [01h 05m 38s]

loss: 0.06430524587631226

Best loss So Far: 0.06430524587631226

Total elapsed time: 02h 41m 36s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary():

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f97e41d6410>

Trial summary

Hyperparameters:

epochs: 60

Score: 0.06430524587631226

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07594212144613266

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08939788863062859

Trial 2:

tuner.search():

Trial 3 Complete [00h 39m 54s]

loss: 0.08899443596601486

Best loss So Far: 0.06491619721055031

Total elapsed time: 02h 16m 04s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary()

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f436ce32e10>

Trial summary

Hyperparameters:

epochs: 60

Score: 0.06491619721055031

Trial summary

Hyperparameters:

epochs: 40

Score: 0.0875425636768341

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08899443596601486

Trial 3:

tuner.search()

Trial 3 Complete [00h 48m 35s]

loss: 0.07456402108073235

Best loss So Far: 0.07456402108073235

Total elapsed time: 02h 06m 02s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary()

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f572171c250>

Trial summary

Hyperparameters:

epochs: 50

Score: 0.07456402108073235

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08825397491455078

Trial summary

Hyperparameters:

epochs: 40

Score: 0.088563472032547


In [ ]:
#Best model : 60 epochs teacher

In [ ]:
teacher_40_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_40_epochs.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_40_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Training the teacher model for 40 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_40_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_40_epochs_dir = os.path.dirname(checkpoint_teacher_model_40_epochs)

In [ ]:
cp_callback_teacher_model_40_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_40_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=40)

In [ ]:
teacher_40_epochs.fit(x_dt_rshp, y_dt_, epochs=40, callbacks=[cp_callback_teacher_model_40_epochs])

Epoch 1/40
3748/3748 [==============================] - 52s 9ms/step - loss: 0.6503 - accuracy: 0.7384
Epoch 2/40
3748/3748 [==============================] - 35s 9ms/step - loss: 0.3782 - accuracy: 0.8513
Epoch 3/40
3748/3748 [==============================] - 36s 10ms/step - loss: 0.3195 - accuracy: 0.8727
Epoch 4/40
3748/3748 [==============================] - 36s 9ms/step - loss: 0.2785 - accuracy: 0.8904
Epoch 5/40
3748/3748 [==============================] - 36s 10ms/step - loss: 0.2479 - accuracy: 0.9024
Epoch 6/40
3748/3748 [==============================] - 36s 9ms/step - loss: 0.2252 - accuracy: 0.9113
Epoch 7/40
3748/3748 [==============================] - 35s 9ms/step - loss: 0.2079 - accuracy: 0.9175
Epoch 8/40
3748/3748 [==============================] - 36s 10ms/step - loss: 0.1959 - accuracy: 0.9226
Epoch 9/40
3748/3748 [==============================] - 36s 10ms/step - loss: 0.1845 - accuracy: 0.9267
Epoch 10/40
3748/3748 [==============================] - 35s 9ms/step

In [ ]:
latest_teacher_40_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_40_epochs_dir)

In [ ]:
print(latest_teacher_40_epochs)
print(checkpoint_teacher_model_40_epochs)

/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt
/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt


In [ ]:
sub_test_data=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [ ]:
sub_test_data.shape

(804, 3001)

In [ ]:
x_test_dt=sub_test_data.drop([3000],axis=1)
y_test_dt=sub_test_data[3000]

In [ ]:
y_test_dt_=np_utils.to_categorical(y_test_dt)

In [ ]:
y_test_dt_

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
x_test_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.031299,-0.065233,0.074522,-0.027280,0.090150,-0.206774,0.006654,-0.084432,0.140158,-0.162570,...,-0.092470,-0.004062,0.038802,-0.030852,-0.022369,-0.033085,-0.023262,-0.080860,0.099526,-0.012992
1,-0.275376,-0.251674,-0.206680,-0.121914,-0.158472,-0.066073,-0.146821,-0.198645,-0.142804,-0.215920,...,-0.000188,0.036771,0.054849,0.039181,0.034361,0.014274,0.049627,0.029942,0.003829,-0.006616
2,0.115511,0.127965,0.159300,0.145641,0.166933,0.172959,0.168540,0.164522,0.164924,0.166531,...,0.022309,0.022710,0.048421,0.068508,0.059268,0.057259,0.068106,0.115913,0.111895,0.114306
3,0.005034,0.014676,0.048823,0.050430,0.053242,0.070517,0.107878,0.115913,0.103057,0.079355,...,0.042395,0.014274,0.003025,-0.015454,-0.016660,-0.026703,-0.019070,-0.025096,-0.019070,-0.007018
4,-0.174179,-0.127743,-0.104079,-0.063893,-0.054070,-0.008081,-0.053624,-0.021922,-0.059428,-0.045140,...,0.041928,0.064253,0.008440,0.001742,-0.017904,-0.041568,-0.045140,-0.043801,0.017370,0.043267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,-0.023708,-0.058982,-0.042015,-0.048712,-0.033085,-0.062554,-0.059875,-0.037103,-0.004062,-0.060768,...,0.000403,-0.000937,0.073183,0.121405,0.162037,0.216956,0.171413,0.152660,0.054430,0.014244
800,-0.004062,-0.054070,-0.033085,-0.015225,0.038802,-0.014778,0.017370,-0.008081,-0.050945,-0.078181,...,-0.114348,-0.111223,-0.134887,-0.038889,-0.092916,-0.079521,-0.048266,0.036123,-0.035764,-0.094702
801,0.045500,-0.062554,0.040142,-0.128190,0.001296,0.005314,0.007547,-0.002723,0.026747,0.064699,...,0.035677,0.102205,0.162930,0.080773,0.079434,0.204008,0.151767,0.128103,0.173646,0.377251
802,0.093817,0.097433,0.089800,0.080158,0.072525,0.046814,0.008248,0.033557,0.062482,0.026326,...,0.078551,0.102655,0.081364,0.058866,0.050430,0.066499,0.083774,0.096228,0.102254,0.088595


In [ ]:
x_test_dt.shape

(804, 3000)

In [ ]:
x_test_dt_rshp=np.array(x_test_dt).reshape(804,3000,1)

In [ ]:
x_test_dt_rshp

array([[[-0.03129874],
        [-0.0652329 ],
        [ 0.07452226],
        ...,
        [-0.08086047],
        [ 0.09952638],
        [-0.01299215]],

       [[-0.27537634],
        [-0.25167404],
        [-0.20667982],
        ...,
        [ 0.02994156],
        [ 0.00382884],
        [-0.00661624]],

       [[ 0.11551092],
        [ 0.12796468],
        [ 0.15929993],
        ...,
        [ 0.11591265],
        [ 0.11189531],
        [ 0.11430572]],

       ...,

       [[ 0.04549962],
        [-0.06255388],
        [ 0.0401416 ],
        ...,
        [ 0.12810252],
        [ 0.17364574],
        [ 0.37725068]],

       [[ 0.09381728],
        [ 0.09743289],
        [ 0.08979994],
        ...,
        [ 0.09622768],
        [ 0.1022537 ],
        [ 0.08859474]],

       [[ 0.02029994],
        [ 0.01266699],
        [ 0.03717277],
        ...,
        [-0.01103532],
        [-0.03072029],
        [-0.04076364]]])

In [ ]:
accuracy_teacher_model_40_epochs = teacher_40_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 1s 8ms/step - loss: 1.0527 - accuracy: 0.8060


Training the teacher model for 50 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_50_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_50_epochs_dir = os.path.dirname(checkpoint_teacher_model_50_epochs)

In [ ]:
cp_callback_teacher_model_50_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_50_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=50)

In [ ]:
teacher_50_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_50_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_50_epochs.fit(x_dt_rshp, y_dt_, epochs=50, callbacks=[cp_callback_teacher_model_50_epochs])

Epoch 1/50
3748/3748 [==============================] - 50s 9ms/step - loss: 0.6485 - accuracy: 0.7392
Epoch 2/50
3748/3748 [==============================] - 33s 9ms/step - loss: 0.3768 - accuracy: 0.8513
Epoch 3/50
3748/3748 [==============================] - 34s 9ms/step - loss: 0.3189 - accuracy: 0.8735
Epoch 4/50
3748/3748 [==============================] - 34s 9ms/step - loss: 0.2791 - accuracy: 0.8904
Epoch 5/50
3748/3748 [==============================] - 34s 9ms/step - loss: 0.2484 - accuracy: 0.9024
Epoch 6/50
3748/3748 [==============================] - 34s 9ms/step - loss: 0.2241 - accuracy: 0.9112
Epoch 7/50
3748/3748 [==============================] - 34s 9ms/step - loss: 0.2066 - accuracy: 0.9182
Epoch 8/50
3748/3748 [==============================] - 34s 9ms/step - loss: 0.1938 - accuracy: 0.9226
Epoch 9/50
3748/3748 [==============================] - 34s 9ms/step - loss: 0.1842 - accuracy: 0.9264
Epoch 10/50
3748/3748 [==============================] - 34s 9ms/step - l

In [ ]:
latest_teacher_50_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_50_epochs_dir)

In [ ]:
accuracy_teacher_model_50_epochs = teacher_50_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 1s 8ms/step - loss: 1.0954 - accuracy: 0.8047


Training the teacher model for 60 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_60_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_60_epochs_dir = os.path.dirname(checkpoint_teacher_model_60_epochs)

In [ ]:
cp_callback_teacher_model_60_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_60_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=60)

In [ ]:
teacher_60_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_60_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_60_epochs.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])

Epoch 1/60
3748/3748 [==============================] - 52s 10ms/step - loss: 0.6494 - accuracy: 0.7385
Epoch 2/60
3748/3748 [==============================] - 36s 10ms/step - loss: 0.3773 - accuracy: 0.8512
Epoch 3/60
3748/3748 [==============================] - 37s 10ms/step - loss: 0.3175 - accuracy: 0.8741
Epoch 4/60
3748/3748 [==============================] - 37s 10ms/step - loss: 0.2767 - accuracy: 0.8912
Epoch 5/60
3748/3748 [==============================] - 36s 10ms/step - loss: 0.2469 - accuracy: 0.9029
Epoch 6/60
3748/3748 [==============================] - 35s 9ms/step - loss: 0.2240 - accuracy: 0.9115
Epoch 7/60
3748/3748 [==============================] - 36s 10ms/step - loss: 0.2079 - accuracy: 0.9176
Epoch 8/60
3748/3748 [==============================] - 37s 10ms/step - loss: 0.1958 - accuracy: 0.9223
Epoch 9/60
3748/3748 [==============================] - 37s 10ms/step - loss: 0.1857 - accuracy: 0.9264
Epoch 10/60
3748/3748 [==============================] - 37s 10ms

In [ ]:
latest_teacher_60_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_60_epochs_dir)

In [ ]:
accuracy_teacher_model_60_epochs = teacher_60_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 2s 8ms/step - loss: 1.0834 - accuracy: 0.7960


training the teacher model for 70 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_70_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/70_epochs_keras_way/teacher_70_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_70_epochs_dir = os.path.dirname(checkpoint_teacher_model_70_epochs)

In [ ]:
cp_callback_teacher_model_70_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_70_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=70)

In [ ]:
teacher_70_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_70_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_70_epochs.fit(x_dt_rshp, y_dt_, epochs=70, callbacks=[cp_callback_teacher_model_70_epochs])

Epoch 1/70
3748/3748 [==============================] - 45s 8ms/step - loss: 0.6494 - accuracy: 0.7383
Epoch 2/70
3748/3748 [==============================] - 29s 8ms/step - loss: 0.3774 - accuracy: 0.8515
Epoch 3/70
3748/3748 [==============================] - 29s 8ms/step - loss: 0.3199 - accuracy: 0.8731
Epoch 4/70
3748/3748 [==============================] - 29s 8ms/step - loss: 0.2827 - accuracy: 0.8878
Epoch 5/70
3748/3748 [==============================] - 29s 8ms/step - loss: 0.2522 - accuracy: 0.9017
Epoch 6/70
3748/3748 [==============================] - 29s 8ms/step - loss: 0.2277 - accuracy: 0.9102
Epoch 7/70
3748/3748 [==============================] - 29s 8ms/step - loss: 0.2106 - accuracy: 0.9164
Epoch 8/70
3748/3748 [==============================] - 29s 8ms/step - loss: 0.1972 - accuracy: 0.9214
Epoch 9/70
3748/3748 [==============================] - 29s 8ms/step - loss: 0.1867 - accuracy: 0.9255
Epoch 10/70
3748/3748 [==============================] - 29s 8ms/step - l

In [ ]:
latest_teacher_70_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_70_epochs_dir)

In [ ]:
accuracy_teacher_model_70_epochs = teacher_70_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 1s 7ms/step - loss: 1.3014 - accuracy: 0.8010


Trial 1

KD using the  epochs teacher model, finalized the teacher with  epochs student model: alpha: 0.9, temp: 5

Trial 2

KD using the  epochs teacher model, finalized the teacher with  epochs student model: alpha: 0.9, temp: 5

Trial 3

KD using the  epochs teacher model, finalized the teacher with  epochs, student model: alpha: 0.9, temp: 5

KD with 40 epochs

In [ ]:
teacher_model=model_b(verbose=VBS) #40 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 12s 7ms/step - loss: 1.0527 - accuracy: 0.8060


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.97674419, 0.3245614 , 0.90909091, 0.71428571, 0.38271605])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.7327995188711096
Recall: 0.6614796524629246
F1 score: 0.677704026505303
              precision    recall  f1-score   support

           0   0.871369  0.976744  0.921053       430
           1   0.425287  0.324561  0.368159       114
           2   0.815217  0.909091  0.859599       165
           3   0.714286  0.714286  0.714286        14
           4   0.837838  0.382716  0.525424        81

    accuracy                       0.805970       804
   macro avg   0.732800  0.661480  0.677704       804
weighted avg   0.790482  0.805970  0.786587       804



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=50)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/50
3748/3748 [==============================] - 33s 8ms/step - categorical_accuracy: 0.6786 - student_loss: 0.7829 - distillation_loss: 0.0016
Epoch 2/50
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7631 - student_loss: 0.5677 - distillation_loss: 0.0012
Epoch 3/50
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7786 - student_loss: 0.5297 - distillation_loss: 0.0011
Epoch 4/50
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7907 - student_loss: 0.5043 - distillation_loss: 0.0011
Epoch 5/50
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8002 - student_loss: 0.4843 - distillation_loss: 0.0010
Epoch 6/50
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8071 - student_loss: 0.4681 - distillation_loss: 9.8167e-04
Epoch 7/50
3748/3748 [==============================] - 29s 8ms/step - categorical_accurac

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7475 - student_loss: 0.7482


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.98604651, 0.09649123, 0.82424242, 1.        , 0.19753086])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.5915212573778145
Recall: 0.6208622056276075
F1 score: 0.5331373048552545
              precision    recall  f1-score   support

           0   0.809160  0.986047  0.888889       430
           1   0.354839  0.096491  0.151724       114
           2   0.719577  0.824242  0.768362       165
           3   0.378378  1.000000  0.549020        14
           4   0.695652  0.197531  0.307692        81

    accuracy                       0.747512       804
   macro avg   0.591521  0.620862  0.533137       804
weighted avg   0.707420  0.747512  0.695159       804



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.98604651, 0.09649123, 0.82424242, 1.        , 0.19753086])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
cm.diagonal()

array([0.98604651, 0.09649123, 0.82424242, 1.        , 0.19753086])

In [ ]:
#50 epochs teacher model

#alpha=0.9, temp=4

#50 epochs: 82.41
#60 epochs: 84.53
#70 epochs: 84.04

#alpha=0.9, temp=5

#50 epochs: 81.43
#60 epochs: 83.88
#70 epochs: 

#alpha=0.9, temp=3

#50 epochs: 83.88
#60 epochs: 83.06


KD using the 50 epochs teacher model

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model=model_b(verbose=VBS) #50 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 12s 7ms/step - loss: 1.0954 - accuracy: 0.8047


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.9744186 , 0.33333333, 0.91515152, 0.71428571, 0.35802469])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.735875558149895
Recall: 0.65904277175595
F1 score: 0.6754950635783562
              precision    recall  f1-score   support

           0   0.869295  0.974419  0.918860       430
           1   0.408602  0.333333  0.367150       114
           2   0.834254  0.915152  0.872832       165
           3   0.714286  0.714286  0.714286        14
           4   0.852941  0.358025  0.504348        81

    accuracy                       0.804726       804
   macro avg   0.735876  0.659043  0.675495       804
weighted avg   0.792435  0.804726  0.785863       804



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=60)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/60
3748/3748 [==============================] - 34s 8ms/step - categorical_accuracy: 0.6796 - student_loss: 0.7800 - distillation_loss: 0.0017
Epoch 2/60
3748/3748 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7613 - student_loss: 0.5683 - distillation_loss: 0.0012
Epoch 3/60
3748/3748 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7760 - student_loss: 0.5345 - distillation_loss: 0.0012
Epoch 4/60
3748/3748 [==============================] - 31s 8ms/step - categorical_accuracy: 0.7879 - student_loss: 0.5127 - distillation_loss: 0.0011
Epoch 5/60
3748/3748 [==============================] - 31s 8ms/step - categorical_accuracy: 0.7955 - student_loss: 0.4964 - distillation_loss: 0.0011
Epoch 6/60
3748/3748 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8031 - student_loss: 0.4820 - distillation_loss: 0.0010
Epoch 7/60
3748/3748 [==============================] - 30s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7624 - student_loss: 0.6406


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.96744186, 0.21052632, 0.84848485, 0.92857143, 0.24691358])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.6187811124135897
Recall: 0.6403876067115561
F1 score: 0.5829833091351129
              precision    recall  f1-score   support

           0   0.845528  0.967442  0.902386       430
           1   0.400000  0.210526  0.275862       114
           2   0.725389  0.848485  0.782123       165
           3   0.433333  0.928571  0.590909        14
           4   0.689655  0.246914  0.363636        81

    accuracy                       0.762438       804
   macro avg   0.618781  0.640388  0.582983       804
weighted avg   0.734820  0.762438  0.729169       804



In [ ]:
#60 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #60 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 12s 7ms/step - loss: 1.0834 - accuracy: 0.7960


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.9372093 , 0.36842105, 0.87878788, 0.71428571, 0.49382716])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.7190161658456551
Recall: 0.6785062217049161
F1 score: 0.6937760664012526
              precision    recall  f1-score   support

           0   0.897550  0.937209  0.916951       430
           1   0.381818  0.368421  0.375000       114
           2   0.819209  0.878788  0.847953       165
           3   0.769231  0.714286  0.740741        14
           4   0.727273  0.493827  0.588235        81

    accuracy                       0.796020       804
   macro avg   0.719016  0.678506  0.693776       804
weighted avg   0.788957  0.796020  0.789762       804



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=70)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/70
3748/3748 [==============================] - 33s 8ms/step - categorical_accuracy: 0.7046 - student_loss: 0.7276 - distillation_loss: 0.0016
Epoch 2/70
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7968 - student_loss: 0.4985 - distillation_loss: 0.0011
Epoch 3/70
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8210 - student_loss: 0.4413 - distillation_loss: 9.4985e-04
Epoch 4/70
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8316 - student_loss: 0.4159 - distillation_loss: 8.9640e-04
Epoch 5/70
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8384 - student_loss: 0.4030 - distillation_loss: 8.6571e-04
Epoch 6/70
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8439 - student_loss: 0.3925 - distillation_loss: 8.4129e-04
Epoch 7/70
3748/3748 [==============================] - 29s 8ms/step - categor

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7749 - student_loss: 0.5889


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.98139535, 0.22807018, 0.82424242, 0.92857143, 0.32098765])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.6470137223660507
Recall: 0.6566534062821293
F1 score: 0.6053842048418402
              precision    recall  f1-score   support

           0   0.824219  0.981395  0.895966       430
           1   0.412698  0.228070  0.293785       114
           2   0.839506  0.824242  0.831804       165
           3   0.393939  0.928571  0.553191        14
           4   0.764706  0.320988  0.452174        81

    accuracy                       0.774876       804
   macro avg   0.647014  0.656653  0.605384       804
weighted avg   0.755518  0.774876  0.746736       804



In [ ]:
#40 epochs teacher model
#alpha=0.9, temp=4
#40 epochs: 83.71
#50 epochs: 84.36
#60 epochs: 83.88

In [ ]:
#70 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #70 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/70_epochs_keras_way/teacher_70_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 12s 7ms/step - loss: 1.3014 - accuracy: 0.8010


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.96046512, 0.35087719, 0.92121212, 0.78571429, 0.34567901])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.7326324051324051
Recall: 0.6727895457067223
F1 score: 0.6777619262790736
              precision    recall  f1-score   support

           0   0.875000  0.960465  0.915743       430
           1   0.404040  0.350877  0.375587       114
           2   0.821622  0.921212  0.868571       165
           3   0.687500  0.785714  0.733333        14
           4   0.875000  0.345679  0.495575        81

    accuracy                       0.800995       804
   macro avg   0.732632  0.672790  0.677762       804
weighted avg   0.794003  0.800995  0.783966       804



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=80)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/80
3748/3748 [==============================] - 35s 8ms/step - categorical_accuracy: 0.7092 - student_loss: 0.7185 - distillation_loss: 0.0016
Epoch 2/80
3748/3748 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8027 - student_loss: 0.4830 - distillation_loss: 0.0011
Epoch 3/80
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8222 - student_loss: 0.4352 - distillation_loss: 9.5743e-04
Epoch 4/80
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8338 - student_loss: 0.4116 - distillation_loss: 9.0541e-04
Epoch 5/80
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8394 - student_loss: 0.3998 - distillation_loss: 8.7783e-04
Epoch 6/80
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8437 - student_loss: 0.3916 - distillation_loss: 8.5839e-04
Epoch 7/80
3748/3748 [==============================] - 29s 8ms/step - categor

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 0s 4ms/step - categorical_accuracy: 0.7587 - student_loss: 0.6204


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.98837209, 0.24561404, 0.81818182, 0.71428571, 0.14814815])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.6405966815224049
Recall: 0.5829203617453311
F1 score: 0.5577721370270992
              precision    recall  f1-score   support

           0   0.826848  0.988372  0.900424       430
           1   0.318182  0.245614  0.277228       114
           2   0.823171  0.818182  0.820669       165
           3   0.434783  0.714286  0.540541        14
           4   0.800000  0.148148  0.250000        81

    accuracy                       0.758706       804
   macro avg   0.640597  0.582920  0.557772       804
weighted avg   0.744437  0.758706  0.723898       804



Trying out diff teachers and different students for perspective(the teacher who's epochs had been tuned)

In [ ]:
final_teacher_model=model_b(verbose=VBS) #50 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
final_teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
final_teacher_model.fit(x_dt_rshp, y_dt_, epochs=50)

Epoch 1/50
3748/3748 [==============================] - 44s 8ms/step - loss: 0.6513 - accuracy: 0.7382
Epoch 2/50
3748/3748 [==============================] - 29s 8ms/step - loss: 0.3793 - accuracy: 0.8509
Epoch 3/50
3748/3748 [==============================] - 29s 8ms/step - loss: 0.3190 - accuracy: 0.8737
Epoch 4/50
3748/3748 [==============================] - 29s 8ms/step - loss: 0.2784 - accuracy: 0.8908
Epoch 5/50
3748/3748 [==============================] - 29s 8ms/step - loss: 0.2489 - accuracy: 0.9023
Epoch 6/50
3748/3748 [==============================] - 28s 8ms/step - loss: 0.2252 - accuracy: 0.9107
Epoch 7/50
3748/3748 [==============================] - 28s 8ms/step - loss: 0.2080 - accuracy: 0.9176
Epoch 8/50
3748/3748 [==============================] - 28s 8ms/step - loss: 0.1955 - accuracy: 0.9222
Epoch 9/50
3748/3748 [==============================] - 29s 8ms/step - loss: 0.1863 - accuracy: 0.9258
Epoch 10/50
3748/3748 [==============================] - 29s 8ms/step - l

In [ ]:
accuracy_final_teacher_model = final_teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 1s 7ms/step - loss: 1.0987 - accuracy: 0.7848


In [ ]:
preds_final_teacher_model = final_teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_teacher_model_fin=np.argmax(preds_final_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_teacher = confusion_matrix(y_test_dt, preds_final_teacher_model_fin)
matrix_final_teacher.diagonal()/matrix_final_teacher.sum(axis=1)

array([0.93488372, 0.40350877, 0.83030303, 0.92857143, 0.40740741])

In [ ]:
matrix_final_teacher

array([[402,  25,   1,   0,   2],
       [ 46,  46,  18,   0,   4],
       [  0,  17, 137,   8,   3],
       [  0,   0,   1,  13,   0],
       [  7,  34,   7,   0,  33]])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_teacher = f1_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
prec_final_teacher = precision_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
recall_final_teacher = recall_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_teacher)) 
    print("Recall: {}".format(recall_final_teacher))
    print("F1 score: {}".format(f1_final_teacher))

    report_final_teacher = classification_report(y_test_dt, preds_final_teacher_model_fin, digits=6)
    print(report_final_teacher)

Precision: 0.7001386844529587
Recall: 0.7009348718283848
F1 score: 0.6821148682353858
              precision    recall  f1-score   support

           0   0.883516  0.934884  0.908475       430
           1   0.377049  0.403509  0.389831       114
           2   0.835366  0.830303  0.832827       165
           3   0.619048  0.928571  0.742857        14
           4   0.785714  0.407407  0.536585        81

    accuracy                       0.784826       804
   macro avg   0.700139  0.700935  0.682115       804
weighted avg   0.787364  0.784826  0.779060       804



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
#student_model=get_model_cnn()
final_student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:

# Initialize and compile distiller
distiller = Distiller(student=final_student_model, teacher=final_teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=60)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/60
3748/3748 [==============================] - 32s 8ms/step - categorical_accuracy: 0.6684 - student_loss: 0.8115 - distillation_loss: 0.0017
Epoch 2/60
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7531 - student_loss: 0.5906 - distillation_loss: 0.0013
Epoch 3/60
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7748 - student_loss: 0.5376 - distillation_loss: 0.0012
Epoch 4/60
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7899 - student_loss: 0.5041 - distillation_loss: 0.0011
Epoch 5/60
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7991 - student_loss: 0.4841 - distillation_loss: 0.0011
Epoch 6/60
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8074 - student_loss: 0.4669 - distillation_loss: 0.0010
Epoch 7/60
3748/3748 [==============================] - 29s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

26/26 [==============================] - 0s 3ms/step - categorical_accuracy: 0.7649 - student_loss: 0.6609


In [ ]:
preds_final_student_model = final_student_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_student_model_fin=np.argmax(preds_final_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_student = confusion_matrix(y_test_dt, preds_final_student_model_fin)
matrix_final_student.diagonal()/matrix_final_student.sum(axis=1)

array([0.98372093, 0.20175439, 0.87272727, 0.85714286, 0.16049383])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_student = f1_score(y_test_dt, preds_final_student_model_fin, average="macro")
prec_final_student = precision_score(y_test_dt, preds_final_student_model_fin, average="macro")
recall_final_student = recall_score(y_test_dt, preds_final_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_student)) 
    print("Recall: {}".format(recall_final_student))
    print("F1 score: {}".format(f1_final_student))

    report_final_student = classification_report(y_test_dt, preds_final_student_model_fin, digits=6)
    print(report_final_student)

Precision: 0.6121232401030657
Recall: 0.6151678546456189
F1 score: 0.5617798820426989
              precision    recall  f1-score   support

           0   0.834320  0.983721  0.902882       430
           1   0.389831  0.201754  0.265896       114
           2   0.757895  0.872727  0.811268       165
           3   0.428571  0.857143  0.571429        14
           4   0.650000  0.160494  0.257426        81

    accuracy                       0.764925       804
   macro avg   0.612123  0.615168  0.561780       804
weighted avg   0.729976  0.764925  0.722962       804

